In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime

# Load in all data
### Read in data from database and configure column types.

In [2]:
# URL and header for Parse
base_url = 'https://dtr-les.herokuapp.com/parse/classes/'
header = {'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'}
data = {'limit': '1000'}

# hotspots
resp = requests.get(base_url + 'hotspot', headers=header, data=data)
hotspots = pd.DataFrame(resp.json()['results'])

hotspot_string_cols = ['objectId', 'vendorId', 'archiver', 'locationCommonName', \
                       'tag', 'submissionMethod', 'beaconId']
hotspots['createdAt'] = (pd.to_datetime(hotspots['createdAt']).astype(np.int64) // 10**9) + hotspots['gmtOffset']
hotspots['updatedAt'] = (pd.to_datetime(hotspots['updatedAt']).astype(np.int64) // 10**9) + hotspots['gmtOffset']
hotspots[hotspot_string_cols] = hotspots[hotspot_string_cols].astype(str)

# sent notifications
resp = requests.get(base_url + 'notificationSent', headers=header, data=data)
notification_sent = pd.DataFrame(resp.json()['results'])

notification_sent_string_cols = ['objectId', 'vendorId', 'hotspotId', 'notificationString']
notification_sent['createdAt'] = (pd.to_datetime(notification_sent['createdAt']).astype(np.int64) // 10**9) + \
                                  notification_sent['gmtOffset']
notification_sent['updatedAt'] = (pd.to_datetime(notification_sent['updatedAt']).astype(np.int64) // 10**9) + \
                                  notification_sent['gmtOffset']
notification_sent[notification_sent_string_cols] = notification_sent[notification_sent_string_cols].astype(str)

# notification responses
resp = requests.get(base_url + 'pingResponse', headers=header, data=data)
notification_responses = pd.DataFrame(resp.json()['results'])

notification_responses_string_cols = ['objectId', 'hotspotId', 'vendorId', 'question', 'response', 'tag']
notification_responses['createdAt'] = (pd.to_datetime(notification_responses['createdAt']).astype(np.int64) \
                                       // 10**9) + notification_responses['gmtOffset']
notification_responses['updatedAt'] = (pd.to_datetime(notification_responses['updatedAt']).astype(np.int64) \
                                       // 10**9) + notification_responses['gmtOffset']
notification_responses[notification_responses_string_cols] = \
    notification_responses[notification_responses_string_cols].astype(str)

# users
resp = requests.get(base_url + 'user', headers=header, data=data)
users = pd.DataFrame(resp.json()['results'])

user_string_cols = ['objectId', 'firstName', 'lastName', 'vendorId', \
                    'firstPreference', 'secondPreference', 'thirdPreference', 'fourthPreference']
users['createdAt'] = (pd.to_datetime(users['createdAt']).astype(np.int64) // 10**9) + (-18000)
users['updatedAt'] = (pd.to_datetime(users['updatedAt']).astype(np.int64) // 10**9) + (-18000)
users[user_string_cols] = users[user_string_cols].astype(str)

# pretracking debug (to see app terminations) 
resp = requests.get(base_url + 'pretracking_debug', headers=header, data=data)
debug = pd.DataFrame(resp.json()['results'])

debug_string_cols = ['objectId', 'vendor_id', 'console_string']
debug['createdAt'] = (pd.to_datetime(debug['createdAt']).astype(np.int64) // 10**9) + (-18000)
debug['updatedAt'] = (pd.to_datetime(debug['updatedAt']).astype(np.int64) // 10**9) + (-18000)
debug[debug_string_cols] = debug[debug_string_cols].astype(str)

### Select data between start and end time of study

In [3]:
# declare start and end times 
start_time_date = pd.to_datetime('02-02-2017 16:00:00')
end_time_date = pd.to_datetime('02-20-2017 00:00:00')

start_time = (start_time_date - datetime.datetime(1970,1,1)).total_seconds()
end_time = (end_time_date - datetime.datetime(1970,1,1)).total_seconds()

# subset each data frame
hotspots = hotspots[(hotspots.createdAt >= start_time) & \
                    (hotspots.createdAt <= end_time)]
notification_sent = notification_sent[(notification_sent.createdAt >= start_time) & \
                                      (notification_sent.createdAt <= end_time) & \
                                      (notification_sent.vendorId != 'F1AE595D-F244-4367-8744-27CA60450F0D')]
notification_responses = notification_responses[(notification_responses.createdAt >= start_time) & \
                                                (notification_responses.createdAt <= end_time) &
                                                (notification_responses.vendorId != 'F1AE595D-F244-4367-8744-27CA60450F0D')]
users = users[(users.createdAt >= (pd.to_datetime('12-01-2016 00:00:00') - datetime.datetime(1970,1,1)).total_seconds()) & \
              (users.createdAt <= end_time) & \
              (users.vendorId != 'F1AE595D-F244-4367-8744-27CA60450F0D')]

debug = debug[(debug.createdAt >= (pd.to_datetime('01-15-2017 00:00:00') - datetime.datetime(1970,1,1)).total_seconds()) & \
              (debug.createdAt <= end_time)]

In [4]:
print(users)

     createdAt firstName firstPreference fourthPreference   lastName  \
6   1484747461                      food       surprising              
7   1480958096      alex            food       surprising   kaldjian   
12  1480933220    Leesha            food            space   Maliakal   
21  1480958630     Katie            food            queue     George   
22  1481118088       Meg           space            queue     Grasse   
28  1480957557    Leesha      surprising            space   Maliakal   
33  1481101695    Alaina            food            queue     Kafkes   
35  1480987577      Ryan      surprising            space     Madden   
38  1480957413    Jennie            food            space     Werner   
41  1480958820   Allison            food       surprising        Sun   
47  1486139561      Greg            food       surprising        Kim   
48  1486139598    Jennie            food       surprising    Werner    
49  1486139739      Josh            food            queue       

In [5]:
print(debug)

                                        console_string   createdAt  \
15                             App entering foreground  1485533221   
33                             App entering background  1484928272   
82                             App entering foreground  1484915390   
102                            App entering background  1484915274   
110                            App entering background  1485533374   
164  The operation couldn’t be completed. (kCLError...  1484861161   
165                            App entering foreground  1485530427   
169  The operation couldn’t be completed. (kCLError...  1484938350   
213  The operation couldn’t be completed. (kCLError...  1485187453   
232                            App entering foreground  1484928441   
284                            App entering background  1485499273   
291                            App entering background  1484915143   
315                            App entering background  1485483056   
334  The operation c

# Compute metrics from study

### Basic demographic info from study

In [6]:
print('Study Length: ' + str((end_time_date - start_time_date)))
print('Number of users: ' + str(len(users)))
print('Male: 2, Female: 6')

Study Length: 17 days 08:00:00
Number of users: 15
Male: 2, Female: 6


In [7]:
print(notification_responses)

     createdAt  gmtOffset   hotspotId    objectId     question  \
52  1486334812     -21600  vE4k73uRkZ  rdjEUjAJ4J      isspace   
54  1486392699     -21600  1Oe3VvnkuA  sFk6DUNdiL  objectright   

                                           response            tag  \
52  com.apple.UNNotificationDefaultActionIdentifier          space   
54                                     I don't know  windowdrawing   

     timestamp   updatedAt                              vendorId  
52  1486356411  1486334812  57DB09A8-068E-4315-B208-D82E24BC7F6D  
54  1486414298  1486392699  57DB09A8-068E-4315-B208-D82E24BC7F6D  


### Number regions, notifications, and responses

In [8]:
kapil_vendor_id = 'F1AE595D-F244-4367-8744-27CA60450F0D'
total_regions_count = len(hotspots)
premarked_regions_count = len(hotspots[(hotspots.locationCommonName != '') | \
                                 (hotspots.vendorId == kapil_vendor_id)])
usermarked_regions_count = total_regions_count - premarked_regions_count

usermarked_regions_agg = hotspots[(hotspots.locationCommonName == '') & \
                                 (hotspots.vendorId != kapil_vendor_id)].groupby('tag').size()

print('Number regions: ' + str(total_regions_count))
print('Number Regions pre-marked: ' + str(premarked_regions_count))
print('Number User-marked regions: ' + str(usermarked_regions_count))
print('Number User-marked regions by category: ' + str(dict(usermarked_regions_agg)))

Number regions: 28
Number Regions pre-marked: 26
Number User-marked regions: 2
Number User-marked regions by category: {'windowdrawing': 2}


In [9]:
invalid_responses = ['com.apple.UNNotificationDefaultActionIdentifier', 
                     'com.apple.UNNotificationDismissActionIdentifier']
cleaned_notification_responses = notification_responses[~notification_responses.response.isin(invalid_responses)]
notifications_responded_to = pd.merge(notification_sent[['hotspotId', 'vendorId']], 
                                      cleaned_notification_responses[['hotspotId', 'vendorId']], how='inner')
notification_merged = pd.merge(notification_sent, cleaned_notification_responses, \
                               on=['hotspotId', 'vendorId'], how='inner')
notification_merged['timediff'] = notification_merged['createdAt_y'] - notification_merged['createdAt_x']

print('Number notifications sent: ' + str(len(notification_sent)))
print('Number notifications responded to: ' + str(len(notifications_responded_to)))
print('Notification response rate: ' + str(len(notifications_responded_to) / len(notification_sent)))
print('Median notification response time in minutes: ' + str(notification_merged['timediff'].median() / 60))

Number notifications sent: 13
Number notifications responded to: 1
Notification response rate: 0.07692307692307693
Median notification response time in minutes: 2.5166666666666666


In [10]:
print(notification_sent)

      createdAt  gmtOffset   hotspotId  \
15   1486392548     -21600  1Oe3VvnkuA   
79   1486136181     -21600  1Oe3VvnkuA   
335  1486136147     -21600  1Oe3VvnkuA   
439  1486249240     -21600  vE4k73uRkZ   
440  1486326320     -21600  vE4k73uRkZ   
441  1486334800     -21600  vE4k73uRkZ   
450  1486573834     -21600  vqcVF4hRDl   
451  1486641403     -21600  u53PY5cXsn   
452  1486659580     -21600  usWlN02wyT   
455  1486731382     -21600  46qQR6CWyW   
456  1486742604     -21600  46qQR6CWyW   
461  1486832498     -21600  YsiYqXbeVo   
462  1487005744     -21600  kRMEZTjy7m   

                                    notificationString    objectId  \
15               Notified for beacon region sJlkSUIKVR  8KY0EorjFD   
79               Notified for beacon region sJlkSUIKVR  TBn5Ug8dIL   
335              Notified for beacon region sJlkSUIKVR  iun37TSbC5   
439  Notified for vE4k73uRkZ (42.05833, -87.683644)...  NOX47qmdxF   
440  Notified for vE4k73uRkZ (42.05833, -87.683644)...  CHeBH

In [11]:
print(hotspots)

    archived archiver    beaconId   createdAt  gmtOffset  \
38      True   system              1486720275     -21600   
60      True   system              1487192577     -18000   
61      True   system              1487239457     -18000   
86      True   system              1487145768     -18000   
103     True   system              1486915337     -18000   
133     True   system              1486778519     -18000   
155     True   system              1487099406     -18000   
214     True   system  sJlkSUIKVR  1486472208     -21600   
236     True   system  sJlkSUIKVR  1486512346     -21600   
242     True   system              1486681371     -21600   
269     True   system              1486512172     -18000   
270     True   system              1486733056     -18000   
273     True   system              1486554216     -21600   
279     True   system  sJlkSUIKVR  1486392950     -21600   
299     True   system              1486468907     -18000   
323     True   system              14866

In [12]:
print(notifications_responded_to)

    hotspotId                              vendorId
0  1Oe3VvnkuA  57DB09A8-068E-4315-B208-D82E24BC7F6D


### Compute average percentage of region info filled

In [75]:
def proportion_filled(info_dict):
    """
    Compute the amount of a info_dict that is filled (i.e. value(key) != '')
    
    Input:
        info_dict (dict): dictionary containing info
       
    Output: 
        (float): proportion of info_dict filled
    """
    n_keys = len(info_dict.keys())
    used_keys = 0
    
    for key, value in info_dict.items():
        if value != '':
            used_keys += 1
    
    return used_keys / n_keys

proportions_for_hotspots = [proportion_filled(x) for x in hotspots['info']]
avg_filled = np.average([x for x in proportions_for_hotspots if x > 0])

print('Average number of entries filled in info: ' + str(avg_filled))

Average number of entries filled in info: nan


/Users/kapilgarg/miniconda3/envs/smearenv/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/kapilgarg/miniconda3/envs/smearenv/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [33]:
base_url = 'https://dtr-les.herokuapp.com/parse/functions/'
header = {'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'}
data = {'latitude': 42.063333,
       'longitude':  -87.683778,
       'vendorId': 'F1AE595D-F244-4367-8744-27CA60450F0D'}
resp = requests.post(base_url + 'naivelyRetrieveLocationsForTracking', headers=header, data=data)
print(json.dumps(resp.json()["result"], indent=4, sort_keys=True))

[
    {
        "__type": "Object",
        "archived": false,
        "archiver": "",
        "beaconId": "",
        "className": "hotspot",
        "createdAt": "2017-02-10T05:02:51.382Z",
        "gmtOffset": -21600,
        "info": {
            "colorleft": "",
            "colorright": "",
            "objectleft": "",
            "objectright": "",
            "valueleft": "",
            "valueright": ""
        },
        "location": {
            "__type": "GeoPoint",
            "latitude": 42.05633281268371,
            "longitude": -87.68606565900862
        },
        "locationCommonName": "",
        "objectId": "Q5J9ErZKlq",
        "saveTimeForQuestion": {
            "colorleft": 1486702970,
            "colorright": 1486702970,
            "objectleft": 1486702970,
            "objectright": 1486702970,
            "valueleft": 1486702970,
            "valueright": 1486702970
        },
        "submissionMethod": "today_widget",
        "tag": "windowdrawing",
    

In [39]:
base_url = 'https://dtr-les.herokuapp.com/parse/functions/'
header = {'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'}
data = {'vendorId': 'F1AE595D-F244-4367-8744-27CA60450F0D'}
resp = requests.post(base_url + 'fetchUserProfileData', headers=header, data=data)
print(json.dumps(resp.json()["result"], indent=4, sort_keys=True))

{
    "contributionCount": 16,
    "contributionLocations": [
        {
            "category": "windowdrawing",
            "contributionType": "marked",
            "hotspotId": "46qQR6CWyW",
            "latitude": 42.05698039852274,
            "longitude": -87.67700272620439,
            "timestamp": 1486720274
        },
        {
            "category": "guestevent",
            "contributionType": "marked",
            "hotspotId": "zsdCQc2cHe",
            "latitude": 42.05624836500925,
            "longitude": -87.68599994488775,
            "timestamp": 1486681317
        },
        {
            "category": "windowdrawing",
            "contributionType": "marked",
            "hotspotId": "u43XZxaR8L",
            "latitude": 42.05697147179586,
            "longitude": -87.67696542673528,
            "timestamp": 1486633225
        },
        {
            "category": "windowdrawing",
            "contributionType": "marked",
            "hotspotId": "TdHQyZG9dF",
        

In [7]:
base_url = 'https://dtr-les.herokuapp.com/parse/functions/'
header = {'X-Parse-Application-Id': 'PkngqKtJygU9WiQ1GXM9eC0a17tKmioKKmpWftYr'}
data = {'vendorId': 'F1AE595D-F244-4367-8744-27CA60450F0D'}
resp = requests.post(base_url + 'fetchUserProfileData', headers=header, data=data)
print(resp)

<Response [200]>


In [11]:
import requests
base_url = 'https://crowdcheerdb.herokuapp.com/parse/classes/CurrRunnerLocation/'
header = {'X-Parse-Application-Id': 'QXRTROGsVaRn4a3kw4gaFnHGNOsZxXoZ8ULxwZmf'}
data = {"score":1337, "playerName": "Sean Plott"}
resp = requests.post(base_url, headers=header, data=data)
print(resp)

<Response [201]>


In [14]:
import httplib
connection = httplib.HTTPSConnection('https://crowdcheerdb.herokuapp.com/parse/classes/CurrRunnerLocation', 443)
connection.connect()

gaierror: [Errno 8] nodename nor servname provided, or not known